In [1]:
from collections.abc import Iterable
import numpy as np
import os
import pandas as pd
import pickle

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
dir_data = r"E:\tencent files\chrome Download\Research\DEA\DEA_transport\Data_input"
dir_lca = "city_mc"
lca_data = os.path.join(dir_data, dir_lca)

<IPython.core.display.Javascript object>

In [3]:
year_range = range(2013, 2018)

<IPython.core.display.Javascript object>

In [4]:
socio_eco_data = pd.read_excel(
    dir_data + "\data.xlsx", sheet_name="dea_inputData", index_col=0
)

<IPython.core.display.Javascript object>

In [5]:
socio_eco_data_sort = socio_eco_data.sort_values(["year", "City name"])

<IPython.core.display.Javascript object>

In [6]:
file_name_list = os.listdir(lca_data)

<IPython.core.display.Javascript object>

In [7]:
file_name_dict = {}
for year in year_range:
    file_name_dict[str(year)] = [i for i in file_name_list if str(year) in i]

<IPython.core.display.Javascript object>

In [8]:
def cumulate_LCA(file_name):
    with open(os.path.join(lca_data, file_name), "rb") as file:
        data = pickle.load(file)
        data_sum = data.groupby(data.index).sum().T
    return data_sum

<IPython.core.display.Javascript object>

In [9]:
cumulate_LCA(file_name_list[0])

,FDP,GWP,PMFP
0,511.540157,1352.341050,4.232742
1,523.300751,1902.275772,4.484253
2,508.968382,1647.406190,4.385157
3,713.507554,1864.829718,5.653213
4,604.144307,1676.433065,4.645798
...,...,...,...
995,629.384160,1642.027514,4.021832
996,704.168516,1969.659775,4.205560
997,669.511994,1628.380338,4.888662
998,580.621270,1811.665762,4.573845


<IPython.core.display.Javascript object>

In [10]:
def LCA_impact_City(file_name_list, impact_name, mc_number):
    # 将每个城市对应的LCIA链接在一个字典中
    value_dict = {}
    for i in range(len(file_name_list)):
        data_mc = cumulate_LCA(file_name_list[i])
        data_value = data_mc.loc[mc_number, impact_name]
        value_dict[file_name_list[i][:-11]] = data_value
    value_df = pd.DataFrame(
        value_dict.values(), index=value_dict.keys(), columns=[impact_name]
    )

    return value_df

<IPython.core.display.Javascript object>

In [11]:
def connect_soc_lca(data, file_name_list, impact_name_list, mc_number):
    impact_df_list = []
    for impact_name in impact_name_list:
        lca_df = LCA_impact_City(file_name_list, impact_name, mc_number)
        impact_df_list.append(lca_df)
    impact_df = pd.concat(impact_df_list, axis=1)
    data_total = pd.concat([data, impact_df], axis=1)
    return data_total

<IPython.core.display.Javascript object>

In [12]:
def dea_inout(iteration):
    dea_inout = []
    for year in year_range:
        data = connect_soc_lca(
            data_year[str(year)], file_name_dict[str(year)], indicators, iteration,
        )
        dea_inout.append(data)
    dea_inout_df = pd.concat(dea_inout)
    return dea_inout_df

<IPython.core.display.Javascript object>

In [13]:
data_year = {}
for year in year_range:
    data_year[str(year)] = socio_eco_data_sort.loc[
        socio_eco_data_sort.loc[:, "year"] == year
    ]

<IPython.core.display.Javascript object>

In [14]:
indicators = ["GWP", "FDP", "PMFP"]
iteration = 1000

<IPython.core.display.Javascript object>

In [15]:
%%time
with pd.ExcelWriter(os.path.join(dir_data,"DEA_inout.xlsx")) as writer:
    for i in range(iteration):
        dea_inout(i).to_excel(writer,sheet_name=str(i))

Wall time: 44min 35s


<IPython.core.display.Javascript object>